# NB need to re-run training_data.csv using the custom sentencizer
# Also need to fold in the neural net negation predictions

In [1]:
import pandas as pd
import plotly.express as px 

In [2]:
df = pd.read_csv('training_data.csv'
# , nrows=100
)
df.head(1)

,CareEpisodeID,injuryIllnessDetails,impressionPlan,12_lead_label_found,12_lead_label_negated,oxygen_label_found,oxygen_label_negated,12_lead_label_sentence,oxygen_label_sentence
0,21928955,IHT SMH ED - WYCOMBE CSRU Cardiac monitoring ...,NaN,1,0,0,0,"12 lead - RBBB,",NaN


In [3]:
# Create a separate list of Oxygen columns from Phil's dataset
oxy_cols = [
    'Oxygen Administered Under Intervention',
#    'Oxygen ContraIndication Under Medicines',
   'Oxygen Administered Under Medicines'
]

In [4]:
filtered_cols = [
   'CareEpisodeID',
   '12 lead ECG taken'
   ]
filtered_cols.extend(oxy_cols) # This adds all elements from oxy_cols to the end of filtered_cols
# filtered_cols

In [5]:
og = pd.read_csv("Oxygen and 12 Lead Free Text Training Model v0.1 - 20250227.csv" #"nlp_input.csv"
, usecols = filtered_cols
# , nrows = 100
, encoding_errors='ignore'
)

In [6]:
# Create a new field that flags where any EPR oxygen field was "1" 
og['oxy_in_epr'] = og[oxy_cols].any(axis=1)
og.rename(columns={"12 lead ECG taken":"12_lead_in_epr"}, inplace=True)
og.head(1)

,CareEpisodeID,12_lead_in_epr,Oxygen Administered Under Intervention,Oxygen Administered Under Medicines,oxy_in_epr
0,21928955,1.0,NaN,NaN,False


In [7]:
og.oxy_in_epr.value_counts()

oxy_in_epr
False    37131
True     13425
Name: count, dtype: int64

Create a new column in OG to combine Oxygen Administered fields\
Merge field, and 12-lead button field, with training data output \
Compare training data with button presses

In [8]:
# Make a list of fields that I want in the results data
og_merge_cols = [
 'CareEpisodeID',
 '12_lead_in_epr',
#  'Oxygen Administered Under Intervention',
#  'Oxygen Administered Under Medicines',
 'oxy_in_epr'
 ]

In [9]:
# Merge the og columns into the output df to create the results
res = df.merge(og[og_merge_cols], on='CareEpisodeID', how='left')
res.head(1)

,CareEpisodeID,injuryIllnessDetails,impressionPlan,12_lead_label_found,12_lead_label_negated,oxygen_label_found,oxygen_label_negated,12_lead_label_sentence,oxygen_label_sentence,12_lead_in_epr,oxy_in_epr
0,21928955,IHT SMH ED - WYCOMBE CSRU Cardiac monitoring ...,NaN,1,0,0,0,"12 lead - RBBB,",NaN,1.0,False


In [10]:
graph_cols = [
    '12_lead_in_epr',
    '12_lead_label_found',
    'oxy_in_epr',
    'oxygen_label_found'
    ]
bar_data = res[graph_cols].sum() # MAKE THIS INTO A BAR CHART

In [11]:
# bar_data is a pandas Series with index as labels and values as heights
fig = px.bar(
    x=bar_data.index,   # labels for the x-axis
    y=bar_data.values,  # values for the y-axis
    labels={'x': 'Category', 'y': 'Sum'},  # optional: axis labels
    title='Sum of Columns'
)
fig.show()

In [12]:
res.shape

(55572, 11)

In [13]:
res['12_lead_in_epr'].value_counts()

12_lead_in_epr
1.0    19186
Name: count, dtype: int64

In [14]:
# Find rows where 12-lead not pressed in EPR, but found by code
res.loc[(res['12_lead_in_epr']!=1)&(res['12_lead_label_found']==1),:]

,CareEpisodeID,injuryIllnessDetails,impressionPlan,12_lead_label_found,12_lead_label_negated,oxygen_label_found,oxygen_label_negated,12_lead_label_sentence,oxygen_label_sentence,12_lead_in_epr,oxy_in_epr
2,21929012,"O/A - Met by wife and led to pt, pt lying on f...",1/ Obs 2/ ECG 3/ Transport to FPH ED for furth...,1,0,0,0,12 Lead ECG = Sinus Tachycardia.,NaN,NaN,False
20,21929142,O/A - Met by taxi driver at front entrance of ...,"Consent, Observe, Assess, Convey NRS to UHS, P...",1,0,0,0,unable to gain.unable to conduct abdo assessme...,NaN,NaN,False
28,21929371,O/A- MET BY PT WIFE WHO SHOWED CREW TO WHERE P...,"ASSESS, OBS O2 THERAPY 12LEAD ECG 5MG SALBU...",1,0,1,0,"12 LEAD SINUS TACH ?LBBB,",OBS O2 THERAPY 5MG SALBUTAMOL NEBS 500MCG I...,NaN,False
37,21929469,"Pt found unresponsive, laboured breathing, pal...",Administered analgesia Transported to A+E,1,0,0,0,12 lead shows NSR NEURO- c/o seve...,NaN,NaN,True
42,21929491,crew was met by street guardian and bouncer an...,"obs cspine considered, stretcher to pt, pt ...",1,0,0,0,pt poor hx and unable to establish a good hx ...,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
55536,22393389,"hpc- at roughly 5pm this evening, pt has taken...","poa- assess, most vital refused. pt aggressive...",1,0,0,0,full 12 lead ecg refused.,NaN,NaN,False
55542,22393441,o/a - Greeted by daughter at the front door of...,Assess and perform neurological assessment wit...,1,0,0,0,12 LEAD ECG shows Sinus Bradycardia at 50bpm.,NaN,NaN,False
55554,22393524,O/A Greeted by partner at the door. Patient ...,Observations 12 Lead ECG Respiratory assess...,1,0,0,0,CVS 12 Lead ECG - Sinus rhythm Denies any...,NaN,NaN,False
55557,22393547,On arrival - Pt answered her flat door to the ...,NaN,1,0,0,0,"12 lead ECG showing sinus rhythm,",NaN,NaN,False


In [15]:
# Find rows where Oxygen not pressed in EPR, but found by code
res.loc[(res['oxy_in_epr']!=1)&(res['oxygen_label_found']==1),:]

,CareEpisodeID,injuryIllnessDetails,impressionPlan,12_lead_label_found,12_lead_label_negated,oxygen_label_found,oxygen_label_negated,12_lead_label_sentence,oxygen_label_sentence,12_lead_in_epr,oxy_in_epr
1,21928957,OA/ Crew flagged down by other students. Crew ...,"1. arrive on scene 2. assess, obs, history 3...",0,0,1,1,NaN,"normal o2 sats on room air,",NaN,False
22,21929174,O/a: patient standing in doorway on arrival- a...,"Examine patient respiratory assessment, salb...",0,0,1,0,NaN,treated with salbutamol- wheeze persisted- ipr...,NaN,False
28,21929371,O/A- MET BY PT WIFE WHO SHOWED CREW TO WHERE P...,"ASSESS, OBS O2 THERAPY 12LEAD ECG 5MG SALBU...",1,0,1,0,"12 LEAD SINUS TACH ?LBBB,",OBS O2 THERAPY 5MG SALBUTAMOL NEBS 500MCG I...,NaN,False
40,21929487,O/A Met by officer outside station and direct...,"Assess, obs, ECG, convey to ED for further ass...",0,0,1,1,NaN,Crew did not administer further supplementary...,NaN,False
58,21929731,O/a: P/t flat on back on bed. O/e: Airway cl...,NaN,0,0,1,0,NaN,When O2 removed sats came down to 80 but Jenni...,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
55548,22393478,"OA// crew arrived, buzzed out of hours door ...",NaN,0,0,1,0,NaN,HR/RR/BP/O2 between normal ranges PLAN// ...,NaN,False
55552,22393520,"OA// crew greeted by fire service, handover g...",#NAME?,0,0,1,1,NaN,- pt was extremely adamant for not going into...,NaN,False
55563,22393649,O/A - Crew knock on front door and ring door b...,NaN,0,0,1,0,NaN,RR and o2 sats WNL.,NaN,False
55565,22393654,"HX - overdose 10 days ago, today took overdose...","assess, obs, convey to QAh",0,0,1,0,NaN,oxygen saturations normal C- normotensive ...,NaN,False


In [16]:
# Output to csv for checking
# res.to_csv("epr_ner_output_check_250519.csv", index=False)